In [1]:

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import inspect
from tqdm import tqdm
import numpy as np
# Set batch size for training and validation
batch_size = 32


In [2]:
model_dictionary = {m[0]:m[1] for m in inspect.getmembers(tf.keras.applications, inspect.isfunction)}


In [3]:

model_dictionary


{'DenseNet121': <function tensorflow.python.keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet169': <function tensorflow.python.keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'DenseNet201': <function tensorflow.python.keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)>,
 'EfficientNetB0': <function tensorflow.python.keras.applications.efficientnet.EfficientNetB0(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000, classifier_activation='softmax', **kwargs)>,
 'EfficientNetB1': <function tensorflow.python.keras.applications.efficientnet.EfficientNetB1(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=

In [4]:
img_height = 224
img_width = 224
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train/",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  "./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Validation/",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



Found 520 files belonging to 2 classes.
Using 104 files for validation.
Found 130 files belonging to 2 classes.
Using 104 files for training.


In [7]:
model_benchmarks = {'model_name': [], 'num_model_params': [], 'validation_accuracy': []}
for model_name, model in tqdm(model_dictionary.items()):
    # Special handling for "NASNetLarge" since it requires input images with size (331,331)
    if 'NASNetLarge' in model_name:
        input_shape=(331,331,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("nasa")
    else:
        input_shape=(224,224,3)
        train_processed = train_ds
        validation_processed = val_ds
        print("others")
    base_model = model(include_top=False,input_shape=input_shape)
    base_model.trainable = False
    print("models loaded")
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(len(train_ds.class_names))
    model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Resizing(input_shape[0], input_shape[1],interpolation='bilinear', name=None),
        tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
        base_model,
        global_average_layer,
        tf.keras.layers.Dense(10),
        # tf.keras.layers.Dense(10),
        prediction_layer
    ])
    # model.summary()
    base_learning_rate = 0.001
    model.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
    # We can evaluate the model right now to see how it does before training it on our new images
    initial_epochs = 1
    # validation_steps=20

    # loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)
    history = model.fit( train_ds ,
    epochs=initial_epochs,
    shuffle=True,
    batch_size=32,
    validation_data=val_ds,
    )
    model_benchmarks['model_name'].append(model_name)
    model_benchmarks['num_model_params'].append(base_model.count_params())
    model_benchmarks['validation_accuracy'].append(history.history)
#     model_benchmarks['accuracy'].append(history.history['accuracy'][-1])


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

others
models loaded
4/4 [==============================] - 17s 2s/step - loss: 1.8049 - accuracy: 0.3721 - val_loss: 0.8380 - val_accuracy: 0.7885


  4%|██▉                                                                                | 1/28 [00:19<08:51, 19.67s/it]

others
models loaded
4/4 [==============================] - 17s 2s/step - loss: 2.0056 - accuracy: 0.3610 - val_loss: 0.8895 - val_accuracy: 0.7885


  7%|█████▉                                                                             | 2/28 [00:41<08:57, 20.67s/it]

...

 93%|████████████████████████████████████████████████████████████████████████████▏     | 26/28 [14:38<00:47, 23.60s/it]

others
models loaded
4/4 [==============================] - 16s 4s/step - loss: 0.6599 - accuracy: 0.5636 - val_loss: 0.5272 - val_accuracy: 0.7885


 96%|███████████████████████████████████████████████████████████████████████████████   | 27/28 [14:56<00:22, 22.11s/it]

others
models loaded
4/4 [==============================] - 23s 4s/step - loss: 1.0209 - accuracy: 0.4888 - val_loss: 0.8201 - val_accuracy: 0.7885


100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [15:23<00:00, 32.97s/it]


In [8]:

len(train_ds.class_names)


2

In [9]:
# # custom modifications on top of pre-trained model
# clf_model = tf.keras.models.Sequential()
# clf_model.add(pre_trained_model)
# clf_model.add(tf.keras.layers.Dense(len(train_ds.class_names), activation='softmax'))
# clf_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
# history = clf_model.fit(train_processed, epochs=3, validation_data=validation_processed,steps_per_epoch = 15)

#    # Calculate all relevant metrics
# model_benchmarks['model_name'].append(model_name)
# model_benchmarks['num_model_params'].append(pre_trained_model.count_params())
# model_benchmarks['validation_accuracy'].append(history.history['val_accuracy'][-1])


NameError: name 'pre_trained_model' is not defined

In [10]:
model_benchmarks

{'model_name': ['DenseNet121',
  'DenseNet169',
  'DenseNet201',
  'EfficientNetB0',
  'EfficientNetB1',
  'EfficientNetB2',
  'EfficientNetB3',
  'EfficientNetB4',
  'EfficientNetB5',
  'EfficientNetB6',
  'EfficientNetB7',
  'InceptionResNetV2',
  'InceptionV3',
  'MobileNet',
  'MobileNetV2',
  'MobileNetV3Large',
  'MobileNetV3Small',
  'NASNetLarge',
  'NASNetMobile',
  'ResNet101',
  'ResNet101V2',
  'ResNet152',
  'ResNet152V2',
  'ResNet50',
  'ResNet50V2',
  'VGG16',
  'VGG19',
  'Xception'],
 'num_model_params': [7037504,
  12642880,
  18321984,
  4049571,
  6575239,
  7768569,
  10783535,
  17673823,
  28513527,
  40960143,
  64097687,
  54336736,
  21802784,
  3228864,
  2257984,
  4226432,
  1529968,
  84916818,
  4269716,
  42658176,
  42626560,
  58370944,
  58331648,
  23587712,
  23564800,
  14714688,
  20024384,
  20861480],
 'validation_accuracy': [{'loss': [1.450446367263794],
   'accuracy': [0.4615384638309479],
   'val_loss': [0.8380455374717712],
   'val_accuracy

In [8]:

def fun(e,r):
    print(e)
    print(r)
    return(e)

In [57]:
asd=0
IMG_SHAPE=(224,224,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                    include_top=False,
                                                    weights='imagenet')
base_model.summary()

_relu[0][0]        
__________________________________________________________________________________________________
block_6_depthwise (DepthwiseCon (None, 14, 14, 192)  1728        block_6_pad[0][0]                
__________________________________________________________________________________________________
block_6_depthwise_BN (BatchNorm (None, 14, 14, 192)  768         block_6_depthwise[0][0]          
__________________________________________________________________________________________________
block_6_depthwise_relu (ReLU)   (None, 14, 14, 192)  0           block_6_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_6_project (Conv2D)        (None, 14, 14, 64)   12288       block_6_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_6_project_BN (BatchNormal (None, 14, 14, 64)   256         block_6_project[0][0]   

In [10]:
asd= train_ds.map(fun)

Tensor("args_0:0", shape=(None, 224, 224, 3), dtype=float32)
Tensor("args_1:0", shape=(None,), dtype=int32)


In [40]:
a=cv2.imread('./archive/Fundus_Train_Val_Data/Fundus_Scanes_Sorted/Train/Glaucoma_Positive/036.jpg')

In [41]:
feature_exteracted = base_model(np.array([cv2.resize(a,(224,224))]))

In [47]:
np.array(feature_exteracted).shape


(1, 7, 7, 1280)

In [49]:
1*7*7*1280

62720

In [56]:
(224*224*3)/(1*7*7*1280)

2.4